In [2]:
import pandas as pd
import numpy as np
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
import time
import random
import os
from os.path import join
from cd_clustering import *

## 1. Loading data from TCDB and GOA database and merging them:

### (a) Loading database created from TCDB:

In [3]:
df_transporter = pd.read_csv("database.csv")
display(df_transporter.head(2))
print(len(df_transporter))
print("Number of different sequences: %s\
\nNumber of different InChI strings: %s\
\nNumber of different Uniprot IDs: %s" 
      % (len(set(list(df_transporter["Sequence"]))), len(set(list(df_transporter["InChI"]))),len(set(list(df_transporter["UniProt"]))) ))

,ChEBI,TCNumber,Name,UniProt,Sequence,Substrate,InChI
0,CHEBI:1,2.A.22.1.2,SODIUM-DEPENDENTNORADRENALINETRANSPORTER(NOREP...,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,(R)-noradrenaline,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...
1,CHEBI:1,2.A.22.1.6,TransporterOS=SchistosomamansoniGN=Slc6a3PE=2SV=1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,(R)-noradrenaline,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...


10561
Number of different sequences: 6228
Number of different InChI strings: 968
Number of different Uniprot IDs: 6229


### (b) Loading database created from the GOA database:

In [4]:
df_transporter_GOA = pd.read_pickle("df_GOA_Transporter_exp.pkl")
display(df_transporter.head(2))
print(len(df_transporter_GOA))
print("Number of different KEGG IDs: %s\
\nNumber of different Uniprot IDs: %s" 
      % ( len(set(list(df_transporter_GOA["molecule ID"]))),len(set(list(df_transporter_GOA["Uniprot ID"]))) ))

,ChEBI,TCNumber,Name,UniProt,Sequence,Substrate,InChI
0,CHEBI:1,2.A.22.1.2,SODIUM-DEPENDENTNORADRENALINETRANSPORTER(NOREP...,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,(R)-noradrenaline,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...
1,CHEBI:1,2.A.22.1.6,TransporterOS=SchistosomamansoniGN=Slc6a3PE=2SV=1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,(R)-noradrenaline,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...


4613
Number of different KEGG IDs: 273
Number of different Uniprot IDs: 3300


### (c) Checking how many Uniprot IDs are in both datasets:

In [5]:
GOA_UIDs = list(set(df_transporter_GOA["Uniprot ID"]))
TCDB_UIDs = list(set(df_transporter["UniProt"]))
duplicated_UIDs = []

for UID in GOA_UIDs:
    if UID in TCDB_UIDs:
        duplicated_UIDs.append(UID)
len(duplicated_UIDs)

919

### (d) Mapping InChI strings to ECFP vectors and KEGG IDs to ECFP vectors:

In [6]:
df_transporter["ECFP"] = ""
for ind in df_transporter.index:
    mol = Chem.inchi.MolFromInchi(df_transporter["InChI"][ind])
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024).ToBitString()
    df_transporter["ECFP"][ind] = ecfp

In [7]:
mol_folder ="C:\\Users\\alexk\\substrateprediction-main\\data\\mol-files"

df_transporter_GOA["ECFP"] = ""
for ind in df_transporter_GOA.index:
    try:
        mol = Chem.MolFromMolFile(join(mol_folder, df_transporter_GOA["molecule ID"][ind] + '.mol'))
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024).ToBitString()
        df_transporter_GOA["ECFP"][ind] = ecfp
    except:
        pass
    
#Remove all entries without ECFP:
df_transporter_GOA = df_transporter_GOA.loc[df_transporter_GOA["ECFP"] != ""]

### (e) Merging both datasets:

In [8]:
df_transporter["KEGG ID"] = np.nan

for ind in df_transporter_GOA.index:
    seq, uid = df_transporter_GOA["Sequence"][ind], df_transporter_GOA["Uniprot ID"][ind]
    ecfp, kegg_id = df_transporter_GOA["ECFP"][ind], df_transporter_GOA["molecule ID"][ind]
    #check if combination of ECFP and Sequence is already in TCDB:
    help_df = df_transporter.loc[df_transporter["Sequence"] == seq].loc[df_transporter["ECFP"] == ecfp]
    if len(help_df) == 0:
        df_transporter = df_transporter.append({"UniProt" : uid, "Sequence" : seq, "KEGG ID" : kegg_id, "ECFP": ecfp},
                             ignore_index = True)
df_transporter

KeyboardInterrupt: 

In [ ]:
df_transporter.to_csv("database_TCDB_and_GOA.csv", index = False)

In [10]:
df_transporter = pd.read_csv("database_TCDB_and_GOA.csv")

## 2.Spitting dataset into training and test set:
We want to make sure that proteins in the training set and in the test set are not very similar. To be more explicit: There should exist no protein in the training set with a sequence identity score >80% compared to and protein in the test set.

Getting input for cd-hit algorithm:

In [11]:
df_Uniprot = pd.DataFrame({"Uniprot ID" : df_transporter["UniProt"], "Sequence" : df_transporter["Sequence"]})
df_Uniprot.drop_duplicates(inplace = True)
df_Uniprot.reset_index(inplace = True)
df_Uniprot

,index,Uniprot ID,Sequence
0,0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...
1,1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...
2,2,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...
3,3,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...
4,4,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...
...,...,...,...
8423,14092,O61967,MPAFFCLPMACQRQVDSIDRSQSNLQAIPSDIFRFRKLEDLNLTMN...
8424,14093,P00505,MALLHSGRVLPGIAAAFHPGLAAAASARASSWWTHVEMGPPDPILG...
8425,14095,P35396,MEQPQEETPEAREEEKEEVAMGDGAPELNGGPEHTLPSSSCADLSQ...
8426,14096,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...


In [12]:
ofile = open(join(".", "protein_data", 'clusters', "all_sequences.fasta"), "w")
for ind in df_Uniprot.index:
    seq = df_Uniprot["Sequence"][ind]
    if not pd.isnull(seq):
        seq_end = seq.find("#")
        seq = seq[:seq_end]
        ofile.write(">" + str(ind) + "\n" + seq  + "\n")
ofile.close()

In [13]:
df_Uniprot

# cluster the fasta files
cluster_folder = join(".", "protein_data", 'clusters')
start_folder = cluster_folder
cluster_all_levels(start_folder, 
                   cluster_folder, 
                   filename='all_sequences')

In [14]:
###We first cluster in such a way that two different clusters do not contain two enzymes
###with a sequence identity higher than 80%:

# collect cluster members
df_80 = find_cluster_members_80(folder=cluster_folder, 
                          filename='all_sequences')

display(df_80.describe())
display(df_80.head())
display(df_80.tail())



###We first cluster in such a way that two different clusters do not contain two enzymes
###with a sequence identity higher than 60%:

cluster_all_levels_60(start_folder, 
                   cluster_folder, 
                   filename='all_sequences')

# collect cluster members
df_60 = find_cluster_members_60(folder=cluster_folder, 
                       filename='all_sequences')
display(df_60.describe())


###We first cluster in such a way that two different clusters do not contain two enzymes
###with a sequence identity higher than 40%:

# cluster the fasta files
cluster_all_levels(start_folder, 
                   cluster_folder, 
                   filename='all_sequences')

# collect cluster members
df_40 = find_cluster_members(folder=cluster_folder, 
                          filename='all_sequences')

display(df_40.describe())

,cluster
count,8372.000000
mean,3423.562470
std,2088.018396
min,0.000000
25%,1585.750000
50%,3344.500000
75%,5213.250000
max,7180.000000


,cluster,member
0,0,2334
1,1,2449
2,2,2388
3,3,2333
4,4,2489


,cluster,member
8367,7176,5458
8368,7177,4539
8369,7178,4624
8370,7179,3380
8371,7180,2979


,cluster
count,8372.000000
mean,2933.351887
std,1831.179125
min,0.000000
25%,1316.750000
50%,2821.500000
75%,4497.250000
max,6309.000000


,cluster
count,8372.000000
mean,2053.058767
std,1343.434788
min,0.000000
25%,867.000000
50%,1914.500000
75%,3163.250000
max,4659.000000


#### Splitting the dataset in train, validation and test set with a sequence identity cutoff of 80%. Later, we divide the test set in three subparts with identity cutoffs of <40%, 40-60% and 60-80%

In [15]:
df_Uniprot["cluster"] = np.nan
for ind in df_80.index:
    member = int(df_80["member"][ind])
    cluster = df_80["cluster"][ind]
    df_Uniprot["cluster"][member] = cluster

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
clusters = list(set(df_Uniprot["cluster"]))
random.seed(1)
random.shuffle(clusters)
print(len(clusters))

n = int(len(clusters)*0.8)
train_clusters = clusters[:n]
test_clusters = clusters[n:]

training_UIDs = df_Uniprot["Uniprot ID"].loc[df_Uniprot["cluster"].isin(train_clusters)]
test_UIDs = df_Uniprot["Uniprot ID"].loc[df_Uniprot["cluster"].isin(test_clusters)]

df_80["split"] = np.nan
df_80["split"].loc[df_80["cluster"].isin(train_clusters)] = "train"
df_80["split"].loc[df_80["cluster"].isin(test_clusters)] = "test"

train_members = list(df_80["member"].loc[df_80["split"] == "train"])
test_members = list(df_80["member"].loc[df_80["split"] == "test"])

df_60["split"] = np.nan
df_40["split"] = np.nan
df_60["split"].loc[df_60["member"].isin(train_members)] = "train"
df_60["split"].loc[df_60["member"].isin(test_members)] = "test"
df_40["split"].loc[df_40["member"].isin(train_members)] = "train"
df_40["split"].loc[df_40["member"].isin(test_members)] = "test"

len(training_UIDs), len(test_UIDs)

7237


C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


(6729, 1755)

In [17]:
df_UID_MID_train = df_transporter.loc[df_transporter["UniProt"].isin(training_UIDs)]
df_UID_MID_test = df_transporter.loc[df_transporter["UniProt"].isin(test_UIDs)]
len(df_UID_MID_test), len(df_UID_MID_train)

(2965, 11261)

Calculating for every sequence in the validation and test set the maximum accuracy compared to sequences in the training set:

In [ ]:
df_80["identity"] = np.nan
df_80["identity"].loc[df_80["split"].isin(["test"])] =  "60-80%"

test_indices = list(df_80.loc[~pd.isnull(df_80["identity"])].index)


for ind in test_indices:

    member = df_80["member"][ind]
    cluster = list(df_40["cluster"].loc[df_40["member"] == member])[0]
    cluster_splits = list(df_40["split"].loc[df_40["cluster"] == cluster])
    if not "train" in cluster_splits:
        df_80["identity"][ind] = "<40%"
    else:
        cluster = list(df_60["cluster"].loc[df_60["member"] == member])[0]
        cluster_splits = list(df_60["split"].loc[df_60["cluster"] == cluster])
        if not "train" in cluster_splits:
            df_80["identity"][ind] = "40-60%"
            
    if ind % 1000 == 0:
        print(ind)
                    
                    
ind = 0
df_Uniprot["identity"] = np.nan
for ind in df_Uniprot.index:
    try:
        df_Uniprot["identity"][ind] = list(df_80["identity"].loc[df_80["member"] == str(ind)])[0]
    except:
        None
        
df_Uniprot.to_pickle(join(".", "protein_data", "Uniprot_df_with_seq_identities.pkl"))

In [ ]:
df_Uniprot.loc[~pd.isnull(df_Uniprot["identity"])]

## 3.Sampling negative data points:

In [19]:
df_Uniprot = pd.read_pickle(join(".", "protein_data", "Uniprot_df_with_seq_identities.pkl"))

### Find all small molcules (mostly ions):

In [50]:
small_molecules = []
small_ecfps = []

for ind in df_transporter.index:
    ecfp = df_transporter["ECFP"][ind]
    if sum(np.array(list(ecfp)).astype(int)) <=2:
        if not pd.isnull(df_transporter["InChI"][ind]):
            print(df_transporter["InChI"][ind])
            small_molecules.append(df_transporter["InChI"][ind])
            small_ecfps.append(ecfp)
        else:
            print(df_transporter["KEGG ID"][ind])
            small_molecules.append(df_transporter["KEGG ID"][ind])
            small_ecfps.append(ecfp)
small_molecules = list(set(small_molecules))

InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S/Zn/q+2
InChI=1S

InChI=1S/As
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+2
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Fe/q+3
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/q+2
InChI=1S/Mn/

InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S/Ca/q+2
InChI=1S

InChI=1S/Ca/q+2
InChI=1S/Cd/q+2
InChI=1S/Co/q+2
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Ag/q+1
InChI=1S/Al/q+3
InChI=1S/Al/q+3
InChI=1S/Al/q+3
InChI=1S/Al/q+3
InChI=1S/Al/q+3
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S/Cu/q+1
InChI=1S

InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1
InChI=1S/p+1

InChI=1S/Hg
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/Hg/q+2
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/HI/h1H/p-1
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Fe
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/q+2
InChI=1S/Mg/

InChI=1S/H2Se/h1H2
InChI=1S/H2Se/h1H2
InChI=1S/H2Se/h1H2
InChI=1S/Se
InChI=1S/Se
InChI=1S/Si
InChI=1S/Si
InChI=1S/Si
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InChI=1S/Na/q+1
InC

C00001
C00698
C00698
C00001
C00038
C00238
C01330
C01342
C00038
C01330
C01330
C01330
C01330
C01330
C01330
C00001
C01330
C00076
C00238
C00238
C00038
C00238
C00001
C00238
C00001
C00001
C01342
C01330
C00076
C00076
C00238
C00698
C00001
C00238
C00238
C00238
C00080
C00238
C00698
C01330
C00076
C00001
C00001
C00001
C00001
C00001
C00001
C00001
C00238
C00001
C00001
C00698
C00698
C00238
C00238
C00238
C00698
C00076
C00076
C00076
C00238
C00076
C00698
C00001
C00001
C00698
C00698
C00001
C00076
C00001
C00001
C00076
C00698
C00238
C00080
C00076
C00076
C00076
C00076
C00076
C00238
C01330
C00038
C00080
C01342
C01342
C00076
C00034
C00076
C19610
C00698
C00238
C00238
C01330
C00698
C00238
C00034
C00076
C19610
C00238
C00698
C00238
C00698
C00001
C00238
C00034
C00076
C01342
C01330
C01330
C01330
C00001
C00238
C00698
C00238
C00698
C00238
C00238
C00238
C00698
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00238
C00076
C00698
C00698
C00238
C00698
C00698
C00698
C00698
C00698

In [72]:
mol_folder ="C:\\Users\\alexk\\substrateprediction-main\\data\\mol-files\\"

def get_mol(met_ID):
    is_InChI = (met_ID[0:5] == "InChI")  
    if is_InChI:
        try:
            mol = Chem.inchi.MolFromInchi(met_ID)
        except:
            mol = None
        
    else:
        try:
            mol = Chem.MolFromMolFile(mol_folder + met_ID + '.mol')
        except OSError:
            mol = None
            
    return(mol)

def drop_samples_without_mol_file(df):
    droplist = []
    for ind in df.index:
        if get_mol(met_ID = df["molecule ID"][ind]) is None:
            droplist.append(ind)

    df.drop(droplist, inplace = True)
    return(df)

def get_metabolites_and_similarities(df):
    df_metabolites = pd.DataFrame(data = {"ECFP": df["ECFP"], "ID": df["molecule ID"]})
    df_metabolites = df_metabolites.drop_duplicates()
    df_metabolites.reset_index(inplace = True, drop = True)


    ms = [get_mol(met_ID = df_metabolites["ID"][ind]) for ind in df_metabolites.index]
    fps = [Chem.RDKFingerprint(x) for x in ms]

    similarity_matrix = np.zeros((len(ms), len(ms)))
    for i in range(len(ms)):
        for j in range(len(ms)):
            similarity_matrix[i,j] = DataStructs.FingerprintSimilarity(fps[i],fps[j])
            
    return(df_metabolites, similarity_matrix)



def get_valid_list(met_ID, UID, forbidden_metabolites, df_metabolites, similarity_matrix, lower_bound =0.7, upper_bound =0.9):
    binding_met_IDs = list(df_transporter["molecule ID"].loc[df_transporter["UniProt"] == UID])
    k = df_metabolites.loc[df_metabolites["ID"] == met_ID].index[0]

    similarities = similarity_matrix[k,:]
    selection = (similarities< upper_bound) * (similarities >lower_bound) 
    metabolites = list(df_metabolites["ID"].loc[selection])
    
    no_mets = list(set(binding_met_IDs + forbidden_metabolites))
    
    metabolites = [met for met in metabolites if (met not in no_mets)]
    return(metabolites)

def get_valid_list_small(met_ID, UID, allowed_small_molecules):
    binding_met_IDs = list(df_transporter["molecule ID"].loc[df_transporter["UniProt"] == UID])
    metabolites = [met for met in allowed_small_molecules if (met not in binding_met_IDs)]
    return(metabolites)


def create_negative_samples(df, df_metabolites, similarity_matrix):
    start = time.time()
    UID_list = []
    MID_list = []
    forbidden_mets = []

    for ind in df.index:
        if ind % 100 ==0:
            print(ind)
            print("Time: %s [min]" % np.round(float((time.time()-start)/60),2))

            df2 = pd.DataFrame(data = {"Uniprot ID": UID_list, "molecule ID" : MID_list})
            df2["outcome"] = 0
            df = pd.concat([df, df2], ignore_index=True)

            UID_list, MID_list = [], []

            forbidden_mets_old = forbidden_mets.copy()
            all_mets = list(set(df["molecule ID"]))
            all_mets = [met for met in all_mets if not met in forbidden_mets_old]
            forbidden_mets = list(set([met for met in all_mets if 
                                       (np.mean(df["outcome"].loc[df["molecule ID"] == met]) < 1/4)]))
            forbidden_mets = forbidden_mets + forbidden_mets_old
            print(len(forbidden_mets))

        UID = df["Uniprot ID"][ind]
        met_ID = df["molecule ID"][ind]

        metabolites = get_valid_list(met_ID = met_ID, UID = UID, forbidden_metabolites= forbidden_mets,
                                     df_metabolites = df_metabolites, similarity_matrix = similarity_matrix,
                                     lower_bound =0.7, upper_bound =0.95)
        lower_bound = 0.7
        while len(metabolites) < 2:
            lower_bound = lower_bound - 0.2
            metabolites = get_valid_list(met_ID = met_ID, UID = UID, forbidden_metabolites= forbidden_mets,
                                     df_metabolites = df_metabolites, similarity_matrix = similarity_matrix,
                                     lower_bound =lower_bound, upper_bound =0.95)
            if lower_bound <0:
                break
        
        new_metabolites =  random.sample(metabolites, min(3,len(metabolites)))

        for met in new_metabolites:
            UID_list.append(UID), MID_list.append(met)

    df2 = pd.DataFrame(data = {"Uniprot ID": UID_list, "molecule ID" : MID_list})
    df2["outcome"] = 0

    df = pd.concat([df, df2], ignore_index = True)
    return(df)

def create_negative_samples_V2(df, df_metabolites, similarity_matrix):
    start = time.time()
    UID_list = []
    MID_list = []
    forbidden_mets = []

    for ind in df.index:
        if ind % 100 ==0:
            print(ind)
            print("Time: %s [min]" % np.round(float((time.time()-start)/60),2))

            df2 = pd.DataFrame(data = {"Uniprot ID": UID_list, "molecule ID" : MID_list})
            df2["outcome"] = 0
            df = pd.concat([df, df2], ignore_index=True)

            UID_list, MID_list = [], []

            forbidden_mets_old = forbidden_mets.copy()
            all_mets = list(set(df["molecule ID"]))
            all_mets = [met for met in all_mets if not met in forbidden_mets_old]
            forbidden_mets = list(set([met for met in all_mets if 
                                       (np.mean(df["outcome"].loc[df["molecule ID"] == met]) < 1/4)]))
            forbidden_mets = forbidden_mets + forbidden_mets_old
            print(len(forbidden_mets))
            
            allowed_small_molecules = [mol for mol in small_molecules if not mol in forbidden_mets]
            print(len(allowed_small_molecules))

        UID = df["Uniprot ID"][ind]
        met_ID = df["molecule ID"][ind]
        
        #check if we have small molecule:
        if met_ID in small_molecules:
            #sample 2 small molecules:
            metabolites = get_valid_list_small(met_ID = met_ID, UID = UID, allowed_small_molecules = allowed_small_molecules)
            n = min(2,len(metabolites))
            new_metabolites =  random.sample(metabolites, n)
            
            #smale one bigger molecule:
            metabolites = get_valid_list(met_ID = met_ID, UID = UID, forbidden_metabolites= forbidden_mets,
                                         df_metabolites = df_metabolites, similarity_matrix = similarity_matrix,
                                         lower_bound =-0.1, upper_bound =0.95)
            new_metabolites = new_metabolites + list(random.sample(metabolites, 3-n))
            
            for met in new_metabolites:
                UID_list.append(UID), MID_list.append(met)
            
            
        else:
            metabolites = get_valid_list(met_ID = met_ID, UID = UID, forbidden_metabolites= forbidden_mets,
                                         df_metabolites = df_metabolites, similarity_matrix = similarity_matrix,
                                         lower_bound =0.7, upper_bound =0.95)
            lower_bound = 0.7
            while len(metabolites) < 2:
                lower_bound = lower_bound - 0.2
                metabolites = get_valid_list(met_ID = met_ID, UID = UID, forbidden_metabolites= forbidden_mets,
                                         df_metabolites = df_metabolites, similarity_matrix = similarity_matrix,
                                         lower_bound =lower_bound, upper_bound =0.95)
                if lower_bound <0:
                    break

            new_metabolites =  random.sample(metabolites, min(3,len(metabolites)))

            for met in new_metabolites:
                UID_list.append(UID), MID_list.append(met)

    df2 = pd.DataFrame(data = {"Uniprot ID": UID_list, "molecule ID" : MID_list})
    df2["outcome"] = 0

    df = pd.concat([df, df2], ignore_index = True)
    return(df)

In [73]:
df_UID_MID_train = df_transporter.loc[df_transporter["UniProt"].isin(training_UIDs)]
df_UID_MID_test = df_transporter.loc[df_transporter["UniProt"].isin(test_UIDs)]
len(df_UID_MID_test), len(df_UID_MID_train)

(2965, 11261)

In [74]:
df_transporter["molecule ID"] = [df_transporter["InChI"][ind] if not pd.isnull(df_transporter["InChI"][ind])
                                      else df_transporter["KEGG ID"][ind] for ind in df_transporter.index]



df_UID_MID_train["molecule ID"] = [df_UID_MID_train["InChI"][ind] if not pd.isnull(df_UID_MID_train["InChI"][ind])
                                      else df_UID_MID_train["KEGG ID"][ind] for ind in df_UID_MID_train.index]

df_UID_MID_train = pd.DataFrame({"Uniprot ID" : df_UID_MID_train["UniProt"],
                   "Sequence" : df_UID_MID_train["Sequence"],
                  "molecule ID" : df_UID_MID_train["molecule ID"],
                  "ECFP" : df_UID_MID_train["ECFP"],
                  "outcome" : 1})

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


#### Creating negative data points for the training set (experimental evidence):

In [75]:
df_UID_MID_train = drop_samples_without_mol_file(df = df_UID_MID_train)
#calculating similarity matrix for all metabolites in the set:
df_metabolites_train, similarity_matrix_train = get_metabolites_and_similarities(df = df_UID_MID_train)
print(len(df_metabolites_train))

df_UID_MID_train.reset_index(inplace = True, drop = True)

df_UID_MID_train = create_negative_samples_V2(df = df_UID_MID_train, df_metabolites = df_metabolites_train,
                                          similarity_matrix = similarity_matrix_train)
df_UID_MID_train

1116
0
Time: 0.0 [min]
0
79
100
Time: 0.02 [min]
4
76
200
Time: 0.03 [min]
12
73
300
Time: 0.05 [min]
19
73
400
Time: 0.07 [min]
26
73
500
Time: 0.09 [min]
39
73
600
Time: 0.1 [min]
57
72
700
Time: 0.12 [min]
76
71
800
Time: 0.14 [min]
90
71
900
Time: 0.16 [min]
101
71
1000
Time: 0.18 [min]
113
65
1100
Time: 0.2 [min]
126
59
1200
Time: 0.21 [min]
147
59
1300
Time: 0.23 [min]
164
54
1400
Time: 0.25 [min]
180
54
1500
Time: 0.27 [min]
189
53
1600
Time: 0.29 [min]
200
50
1700
Time: 0.3 [min]
207
50
1800
Time: 0.32 [min]
220
50
1900
Time: 0.34 [min]
234
49
2000
Time: 0.36 [min]
243
45
2100
Time: 0.38 [min]
260
45
2200
Time: 0.4 [min]
267
45
2300
Time: 0.42 [min]
276
43
2400
Time: 0.44 [min]
288
43
2500
Time: 0.46 [min]
305
42
2600
Time: 0.48 [min]
311
39
2700
Time: 0.5 [min]
325
38
2800
Time: 0.52 [min]
334
36
2900
Time: 0.55 [min]
344
36
3000
Time: 0.57 [min]
352
34
3100
Time: 0.59 [min]
359
34
3200
Time: 0.61 [min]
377
34
3300
Time: 0.63 [min]
393
34
3400
Time: 0.65 [min]
401
33
3500
Time

,Uniprot ID,Sequence,molecule ID,ECFP,outcome
0,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,0100000000000000001000000000000000000000000000...,1
1,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,InChI=1S/C27H44O/c1-19(2)8-6-9-21(4)25-15-16-2...,0100100010000000000000000000000001011000010000...,1
2,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...,"InChI=1S/C15H20O6/c1-7-3-9-14(5-16,11(19)10(7)...",0100000000000000000000100000000001001000000000...,1
3,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...,InChI=1S/C16H14F3N5O/c1-10(15-14(19)5-20-7-22-...,1100000000000000000000000000000001000000000100...,1
4,WP_068464567.1,MKIKDWNRSLKVRLVGEFFMNTSFWMVFPFLAIYFAEEFGKGLAGM...,InChI=1S/C16H18FN3O3/c1-2-19-9-11(16(22)23)15(...,0100000000000000000100010000000001100010000001...,1
...,...,...,...,...,...
43665,Q0GMA8,NaN,C00238,NaN,0
43666,Q0GMA8,NaN,C00017,NaN,0
43667,Q84W56,NaN,InChI=1S/Mo.4O/q;;;2*-1,NaN,0
43668,Q84W56,NaN,InChI=1S/K/q+1,NaN,0


#### Creating negative data points for the test set

In [76]:
df_UID_MID_test["molecule ID"] = [df_UID_MID_test["InChI"][ind] if not pd.isnull(df_UID_MID_test["InChI"][ind])
                                      else df_UID_MID_test["KEGG ID"][ind] for ind in df_UID_MID_test.index]

df_UID_MID_test = pd.DataFrame({"Uniprot ID" : df_UID_MID_test["UniProt"],
                   "Sequence" : df_UID_MID_test["Sequence"],
                  "molecule ID" : df_UID_MID_test["molecule ID"],
                  "ECFP" : df_UID_MID_test["ECFP"],
                  "outcome" : 1})


df_UID_MID_test = drop_samples_without_mol_file(df = df_UID_MID_test)
#calculating similarity matrix for all metabolites in the set:
df_metabolites_test, similarity_matrix_test = get_metabolites_and_similarities(df = df_UID_MID_test)
print(len(df_metabolites_test))

df_UID_MID_test.reset_index(inplace = True, drop = True)

df_UID_MID_test = create_negative_samples_V2(df = df_UID_MID_test, df_metabolites = df_metabolites_test,
                                          similarity_matrix = similarity_matrix_test)
df_UID_MID_test

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


651
0
Time: 0.0 [min]
0
79
100
Time: 0.01 [min]
16
63
200
Time: 0.02 [min]
32
59
300
Time: 0.02 [min]
54
52
400
Time: 0.03 [min]
76
50
500
Time: 0.04 [min]
89
46
600
Time: 0.05 [min]
109
46
700
Time: 0.06 [min]
125
41
800
Time: 0.07 [min]
144
33
900
Time: 0.07 [min]
167
33
1000
Time: 0.08 [min]
209
33
1100
Time: 0.09 [min]
241
31
1200
Time: 0.1 [min]
260
29
1300
Time: 0.11 [min]
273
25
1400
Time: 0.12 [min]
283
22
1500
Time: 0.13 [min]
319
21
1600
Time: 0.13 [min]
347
20
1700
Time: 0.14 [min]
366
20
1800
Time: 0.15 [min]
385
20
1900
Time: 0.16 [min]
404
19
2000
Time: 0.17 [min]
416
16
2100
Time: 0.18 [min]
442
16
2200
Time: 0.18 [min]
462
15
2300
Time: 0.19 [min]
483
13
2400
Time: 0.2 [min]
505
12
2500
Time: 0.21 [min]
529
12
2600
Time: 0.22 [min]
551
12
2700
Time: 0.23 [min]
572
10
2800
Time: 0.24 [min]
589
10
2900
Time: 0.25 [min]
606
9


,Uniprot ID,Sequence,molecule ID,ECFP,outcome
0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,0100000000000000001000000000000000000000000000...,1
1,V9SBV7,MAMSMAMSKAITARHATHLQHRLVASSSQAAPRLPLLPRRPSLALT...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
2,Q07307,MDNSIHSTDGPDSVIPNSNPKKTVRQRVRLLARHLTTREGLIGDYD...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
3,Q4QG33,MILNFSSLAELYVYTTCVLLGVSMLMPLNALASAPAYMLDYYKYAT...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
4,P42086,MRNGFGKTLSLGIQHVLAMYAGAIVVPLIVGKAMGLTVEQLTYLVS...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
...,...,...,...,...,...
11493,A0A1D8PSH1,NaN,InChI=1S/C34H34N4O4.Fe/c1-7-21-17(3)25-13-26-1...,NaN,0
11494,A0A1D8PSH1,NaN,C00865,NaN,0
11495,Q9CAT6,NaN,"InChI=1S/C3H8O3/c4-1-3(6)2-5/h3-6H,1-2H2",NaN,0
11496,Q9CAT6,NaN,"InChI=1S/C6H13NO3S.Na/c8-11(9,10)7-6-4-2-1-3-5...",NaN,0


Adding ECFPs and Sequence for all newly added data points:

In [77]:
for ind in df_UID_MID_train.index:
    if df_UID_MID_train["outcome"][ind] == 0:
        UID, met_ID = df_UID_MID_train["Uniprot ID"][ind], df_UID_MID_train["molecule ID"][ind]
        df_UID_MID_train["Sequence"][ind] = list(df_transporter["Sequence"].loc[df_transporter["UniProt"] == UID])[0]
        df_UID_MID_train["ECFP"][ind] = list(df_transporter["ECFP"].loc[df_transporter["molecule ID"] == met_ID])[0]
df_UID_MID_train

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Uniprot ID,Sequence,molecule ID,ECFP,outcome
0,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,0100000000000000001000000000000000000000000000...,1
1,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,InChI=1S/C27H44O/c1-19(2)8-6-9-21(4)25-15-16-2...,0100100010000000000000000000000001011000010000...,1
2,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...,"InChI=1S/C15H20O6/c1-7-3-9-14(5-16,11(19)10(7)...",0100000000000000000000100000000001001000000000...,1
3,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...,InChI=1S/C16H14F3N5O/c1-10(15-14(19)5-20-7-22-...,1100000000000000000000000000000001000000000100...,1
4,WP_068464567.1,MKIKDWNRSLKVRLVGEFFMNTSFWMVFPFLAIYFAEEFGKGLAGM...,InChI=1S/C16H18FN3O3/c1-2-19-9-11(16(22)23)15(...,0100000000000000000100010000000001100010000001...,1
...,...,...,...,...,...
43665,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...,C00238,0000000000000000000000000000000000000000000000...,0
43666,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...,C00017,0100000000000000000000000000000000000000000000...,0
43667,Q84W56,MMKPASLQGFSSHASSSIYSDVRRPATTPSKMAAFSALSLCPYTFT...,InChI=1S/Mo.4O/q;;;2*-1,0000000000000000000000000000000000000000000000...,0
43668,Q84W56,MMKPASLQGFSSHASSSIYSDVRRPATTPSKMAAFSALSLCPYTFT...,InChI=1S/K/q+1,0000000000000000000000000000000000000000000000...,0


In [78]:
for ind in df_UID_MID_test.index:
    if df_UID_MID_test["outcome"][ind] == 0:
        UID, met_ID = df_UID_MID_test["Uniprot ID"][ind], df_UID_MID_test["molecule ID"][ind]
        df_UID_MID_test["Sequence"][ind] = list(df_transporter["Sequence"].loc[df_transporter["UniProt"] == UID])[0]
        df_UID_MID_test["ECFP"][ind] = list(df_transporter["ECFP"].loc[df_transporter["molecule ID"] == met_ID])[0]
df_UID_MID_test

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Uniprot ID,Sequence,molecule ID,ECFP,outcome
0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,0100000000000000001000000000000000000000000000...,1
1,V9SBV7,MAMSMAMSKAITARHATHLQHRLVASSSQAAPRLPLLPRRPSLALT...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
2,Q07307,MDNSIHSTDGPDSVIPNSNPKKTVRQRVRLLARHLTTREGLIGDYD...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
3,Q4QG33,MILNFSSLAELYVYTTCVLLGVSMLMPLNALASAPAYMLDYYKYAT...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
4,P42086,MRNGFGKTLSLGIQHVLAMYAGAIVVPLIVGKAMGLTVEQLTYLVS...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1
...,...,...,...,...,...
11493,A0A1D8PSH1,MSSVSSENNSGLFGTDVYDETKENKPKYEHEEGLEFGSDFDFDGEF...,InChI=1S/C34H34N4O4.Fe/c1-7-21-17(3)25-13-26-1...,0000000000000000000000000000000001000000100000...,0
11494,A0A1D8PSH1,MSSVSSENNSGLFGTDVYDETKENKPKYEHEEGLEFGSDFDFDGEF...,C00865,0100000000000000000000000000000000000000000000...,0
11495,Q9CAT6,MEPSKQEVPKLMETPPNISNDSSATEKGEATRQQQLPNNRYALTVD...,"InChI=1S/C3H8O3/c4-1-3(6)2-5/h3-6H,1-2H2",0100000000000000000000000000000000000000000000...,0
11496,Q9CAT6,MEPSKQEVPKLMETPPNISNDSSATEKGEATRQQQLPNNRYALTVD...,"InChI=1S/C6H13NO3S.Na/c8-11(9,10)7-6-4-2-1-3-5...",0010100000000000000000000000000000000000000000...,0


In [79]:
df_UID_MID_train.to_pickle(join(".", "protein_data", "df_UID_MID_train_V2.pkl"))
df_UID_MID_test.to_pickle(join(".", "protein_data", "df_UID_MID_test_V2.pkl"))

## 4.Calculating enzyme representations for alle Sequences:

In [80]:
df_Uniprot.drop_duplicates(inplace = True)
df_Uniprot.reset_index(inplace = True)
df_Uniprot

,level_0,index,Uniprot ID,Sequence,cluster,identity
0,0,0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,1546.0,60-80%
1,1,1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,1049.0,NaN
2,2,2,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,3404.0,NaN
3,3,3,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...,1696.0,NaN
4,4,4,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...,156.0,NaN
...,...,...,...,...,...,...
8423,8423,14092,O61967,MPAFFCLPMACQRQVDSIDRSQSNLQAIPSDIFRFRKLEDLNLTMN...,1167.0,NaN
8424,8424,14093,P00505,MALLHSGRVLPGIAAAFHPGLAAAASARASSWWTHVEMGPPDPILG...,3382.0,NaN
8425,8425,14095,P35396,MEQPQEETPEAREEEKEEVAMGDGAPELNGGPEHTLPSSSCADLSQ...,3257.0,NaN
8426,8426,14096,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...,1896.0,NaN


Creating FASTA file will all sequences as input for ESM1b model

In [95]:
ofile = open(join(".", "protein_data", "all_transporter_sequences.fasta"), "w")
for ind in df_Uniprot.index:
    seq = df_Uniprot["Sequence"][ind]
    if not pd.isnull(seq):
        seq_end = seq.find("#")
        seq = seq[:seq_end]
        ofile.write(">" + str(ind) + "\n" + seq  + "\n")
ofile.close()

Creating enzyme representations on HILBERT. Using the created .pt file to map ESM1b vectors to Sequences:

In [81]:
import torch 

df_Uniprot["ESM1b"] = ""
rep_dict = torch.load(join(".", "protein_data", "all_transporter_sequences.pt"))

for ind in df_Uniprot.index:
    try:
        df_Uniprot["ESM1b"][ind] = rep_dict[str(ind) +".pt"]
    except:
        print(ind)
df_Uniprot

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


4
39
71
77
85
90
94
128
148
149
165
188
207
217
275
316
340
341
342
371
372
441
469
509
510
535
547
591
600
601
625
647
654
783
817
852
889
890
891
892
894
915
930
971
973
974
976
977
979
1020
1021
1057
1063
1082
1092
1178
1179
1181
1182
1193
1207
1237
1406
1408
1497
1519
1558
1561
1562
1565
1571
1634
1636
1647
1666
1677
1688
1696
1698
1707
1717
1720
1727
1744
1745
1754
1757
1761
1762
1764
1767
1771
1780
1781
1789
1811
1823
1824
1845
1853
1854
1856
1858
1867
1869
1872
1979
1987
2002
2068
2074
2085
2134
2155
2191
2198
2199
2201
2213
2217
2220
2221
2229
2232
2233
2239
2240
2241
2243
2245
2249
2256
2262
2268
2269
2270
2272
2273
2274
2275
2284
2290
2291
2293
2296
2297
2298
2300
2305
2312
2319
2322
2323
2333
2334
2336
2338
2347
2363
2371
2372
2376
2377
2378
2382
2387
2388
2389
2391
2397
2404
2405
2408
2410
2413
2417
2419
2423
2428
2429
2431
2432
2433
2435
2437
2449
2451
2459
2460
2462
2465
2466
2467
2468
2474
2476
2477
2480
2482
2485
2489
2490
2491
2493
2496
2502
2507
2509
2511
2512
2515
25

,level_0,index,Uniprot ID,Sequence,cluster,identity,ESM1b
0,0,0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,1546.0,60-80%,"[-0.07108224, 0.096061245, 0.06852332, 0.03081..."
1,1,1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,1049.0,NaN,"[0.03662403, 0.14126705, -0.055604078, 0.01308..."
2,2,2,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,3404.0,NaN,"[-0.074354, 0.2844502, 0.062469013, -0.0139034..."
3,3,3,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...,1696.0,NaN,"[-0.08255166, 0.14464356, 0.08608833, 0.076414..."
4,4,4,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...,156.0,NaN,
...,...,...,...,...,...,...,...
8423,8423,14092,O61967,MPAFFCLPMACQRQVDSIDRSQSNLQAIPSDIFRFRKLEDLNLTMN...,1167.0,NaN,"[-0.010590305, 0.17112774, 0.06293734, -0.0026..."
8424,8424,14093,P00505,MALLHSGRVLPGIAAAFHPGLAAAASARASSWWTHVEMGPPDPILG...,3382.0,NaN,"[-0.064013295, 0.23662551, 0.11381381, 0.00630..."
8425,8425,14095,P35396,MEQPQEETPEAREEEKEEVAMGDGAPELNGGPEHTLPSSSCADLSQ...,3257.0,NaN,"[-0.018252473, 0.25217032, 0.045643948, -0.008..."
8426,8426,14096,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...,1896.0,NaN,"[-0.09520724, 0.0897981, 0.15097311, -0.013405..."


Mapping ESM1b vectors to positive and negative data points:

In [82]:
df_UID_MID_train = df_UID_MID_train.merge(df_Uniprot, how = "left", on = ["Uniprot ID", "Sequence"])
df_UID_MID_test = df_UID_MID_test.merge(df_Uniprot, how = "left", on = ["Uniprot ID", "Sequence"])

In [83]:
#Removing all data points without an ESM1b vector:
df_UID_MID_train = df_UID_MID_train.loc[df_UID_MID_train["ESM1b"] != ""]
df_UID_MID_test = df_UID_MID_test.loc[df_UID_MID_test["ESM1b"] != ""]

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\pandas\core\ops\array_ops.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


#### Mapping sequence idenitity level to all proteins in the test set:

In [84]:
df_Uniprot

,level_0,index,Uniprot ID,Sequence,cluster,identity,ESM1b
0,0,0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,1546.0,60-80%,"[-0.07108224, 0.096061245, 0.06852332, 0.03081..."
1,1,1,E9LD23,MAEESNKNNMTAHLNKINTYKNNLIISNNSINNNNNSINNNNDIID...,1049.0,NaN,"[0.03662403, 0.14126705, -0.055604078, 0.01308..."
2,2,2,P11473,MEAMAASTSLPDPGDFDRNVPRICGVCGDRATGFHFNAMTCEGCKG...,3404.0,NaN,"[-0.074354, 0.2844502, 0.062469013, -0.0139034..."
3,3,3,Q96W86,MTATVPQEGVVDLESQPDDRLRAEALATTAAELPEGYYTSARVMAS...,1696.0,NaN,"[-0.08255166, 0.14464356, 0.08608833, 0.076414..."
4,4,4,F2SG60,MAPTEEANVTKPTGELRPDEKLNYEEDVKCSGSSSTTVGKTAYDTD...,156.0,NaN,
...,...,...,...,...,...,...,...
8423,8423,14092,O61967,MPAFFCLPMACQRQVDSIDRSQSNLQAIPSDIFRFRKLEDLNLTMN...,1167.0,NaN,"[-0.010590305, 0.17112774, 0.06293734, -0.0026..."
8424,8424,14093,P00505,MALLHSGRVLPGIAAAFHPGLAAAASARASSWWTHVEMGPPDPILG...,3382.0,NaN,"[-0.064013295, 0.23662551, 0.11381381, 0.00630..."
8425,8425,14095,P35396,MEQPQEETPEAREEEKEEVAMGDGAPELNGGPEHTLPSSSCADLSQ...,3257.0,NaN,"[-0.018252473, 0.25217032, 0.045643948, -0.008..."
8426,8426,14096,Q0GMA8,MGPPYSDLRESDEDRPAEAVGSVSGSRNALQPLPGEDDEEPFTTYF...,1896.0,NaN,"[-0.09520724, 0.0897981, 0.15097311, -0.013405..."


In [85]:
df_UID_MID_test["Sequence identity"] = np.nan
for ind in df_UID_MID_test.index:
    UID = df_UID_MID_test["Uniprot ID"][ind]
    help_df = df_Uniprot.loc[df_Uniprot["Uniprot ID"] == UID]
    df_UID_MID_test["Sequence identity"][ind] = list(help_df["identity"])[0]

C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [86]:
df_UID_MID_test.head(3)

,Uniprot ID,Sequence,molecule ID,ECFP,outcome,level_0,index,cluster,identity,ESM1b,Sequence identity
0,P23975,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,InChI=1S/C8H11NO3/c9-4-8(12)5-1-2-6(10)7(11)3-...,0100000000000000001000000000000000000000000000...,1,0,0,1546.0,60-80%,"[-0.07108224, 0.096061245, 0.06852332, 0.03081...",60-80%
1,V9SBV7,MAMSMAMSKAITARHATHLQHRLVASSSQAAPRLPLLPRRPSLALT...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1,8,8,2113.0,40-60%,"[0.01073054, 0.13335295, 0.19254746, -0.061254...",40-60%
2,Q07307,MDNSIHSTDGPDSVIPNSNPKKTVRQRVRLLARHLTTREGLIGDYD...,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,0000000000000000000000000000000000000000000000...,1,9,9,1819.0,60-80%,"[0.113539055, 0.16082606, 0.16762808, 0.020850...",60-80%


In [87]:
df_UID_MID_train.to_pickle(join(".", "training_data_V2.pkl"))
df_UID_MID_test.to_pickle(join(".", "test_data_V2.pkl"))

## 5. Trying to fit a first very simple model:

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
df_UID_MID_train = pd.read_pickle(join(".", "training_data_V2.pkl"))
df_UID_MID_test = pd.read_pickle(join(".", "test_data_V2.pkl"))

##### Splitting dataset in 80% training data and 20% test data (splitting by uniprot ID):

In [91]:
train_UIDs = list(set(df_UID_MID_train["Uniprot ID"]))
len(train_UIDs)

6136

In [92]:
df_train = df_UID_MID_train.copy()
df_test = df_UID_MID_test.copy()

In [93]:
from os.path import join
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#from hyperopt import fmin, tpe, hp, Trials, rand
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef



def create_input_and_output_data(df):
    X = ();
    y = ();
    
    for ind in df.index:
        emb = df["ESM1b"][ind]
        ecfp = np.array(list(df["ECFP"][ind])).astype(int)
                
        X = X +(np.concatenate([ecfp, emb]), );
        y = y + (df["outcome"][ind], );

    return(X,y)

train_X, train_y =  create_input_and_output_data(df = df_train)
test_X, test_y =  create_input_and_output_data(df = df_test)


feature_names =  ["ECFP_" + str(i) for i in range(1024)]
feature_names = feature_names + ["ESM1b_" + str(i) for i in range(1280)]

train_X = np.array(train_X)
test_X  = np.array(test_X)

train_y = np.array(train_y)
test_y  = np.array(test_y)

In [101]:
param = {'learning_rate': 0.31553117247348733,
         'max_delta_step': 1.7726044219753656,
         'max_depth': 10,
         'min_child_weight': 1.3845040588450772,
         'num_rounds': 542.68325188584106,
         'reg_alpha': 0.531395259755843,
         'reg_lambda': 3.744980563764689,
         'weight': 0.26187490421514203}

num_round = param["num_rounds"]

param["tree_method"] = "gpu_hist"
param["sampling_method"] = "gradient_based"

param['objective'] = 'binary:logistic'
weights = np.array([param["weight"] if binding == 0 else 1.0 for binding in df_train["outcome"]])

del param["num_rounds"]
del param["weight"]


dtrain = xgb.DMatrix(np.array(train_X), weight = weights, label = np.array(train_y))
dtest = xgb.DMatrix(np.array(test_X), label = np.array(test_y))

evallist = [(dtest, 'eval'), (dtrain, 'train')]

bst = xgb.train(param,  dtrain, int(num_round), evallist)

y_test_pred = np.round(bst.predict(dtest))
acc_test = np.mean(y_test_pred == np.array(test_y))

roc_auc = roc_auc_score(np.array(test_y), bst.predict(dtest))
mcc = matthews_corrcoef(np.array(test_y), y_test_pred)

print("Accuracy on test set: %s, ROC-AUC score for test set: %s, MCC: %s"  % (acc_test, roc_auc, mcc))


[0]	eval-error:0.54359	train-error:0.33881
[1]	eval-error:0.51644	train-error:0.30927
[2]	eval-error:0.47889	train-error:0.27987
[3]	eval-error:0.43384	train-error:0.25327
[4]	eval-error:0.42839	train-error:0.24121
[5]	eval-error:0.41924	train-error:0.23573
[6]	eval-error:0.40835	train-error:0.22345
[7]	eval-error:0.40105	train-error:0.21954
[8]	eval-error:0.38636	train-error:0.20369
[9]	eval-error:0.37644	train-error:0.19678
[10]	eval-error:0.37138	train-error:0.19297
[11]	eval-error:0.36534	train-error:0.18771
[12]	eval-error:0.36116	train-error:0.18304
[13]	eval-error:0.35523	train-error:0.17742
[14]	eval-error:0.34365	train-error:0.16988
[15]	eval-error:0.33800	train-error:0.16530
[16]	eval-error:0.33110	train-error:0.15876
[17]	eval-error:0.32720	train-error:0.15394
[18]	eval-error:0.32302	train-error:0.15004
[19]	eval-error:0.32146	train-error:0.14586
[20]	eval-error:0.31670	train-error:0.14126
[21]	eval-error:0.31349	train-error:0.13914
[22]	eval-error:0.30784	train-error:0.1357

[185]	eval-error:0.17883	train-error:0.02556
[186]	eval-error:0.17815	train-error:0.02553
[187]	eval-error:0.17805	train-error:0.02550
[188]	eval-error:0.17824	train-error:0.02543
[189]	eval-error:0.17795	train-error:0.02527
[190]	eval-error:0.17786	train-error:0.02513
[191]	eval-error:0.17795	train-error:0.02505
[192]	eval-error:0.17805	train-error:0.02494
[193]	eval-error:0.17776	train-error:0.02492
[194]	eval-error:0.17883	train-error:0.02488
[195]	eval-error:0.17834	train-error:0.02476
[196]	eval-error:0.17795	train-error:0.02449
[197]	eval-error:0.17815	train-error:0.02430
[198]	eval-error:0.17854	train-error:0.02433
[199]	eval-error:0.17795	train-error:0.02424
[200]	eval-error:0.17756	train-error:0.02426
[201]	eval-error:0.17698	train-error:0.02417
[202]	eval-error:0.17698	train-error:0.02411
[203]	eval-error:0.17756	train-error:0.02401
[204]	eval-error:0.17747	train-error:0.02389
[205]	eval-error:0.17717	train-error:0.02387
[206]	eval-error:0.17766	train-error:0.02400
[207]	eval

[368]	eval-error:0.16180	train-error:0.01808
[369]	eval-error:0.16239	train-error:0.01812
[370]	eval-error:0.16239	train-error:0.01811
[371]	eval-error:0.16219	train-error:0.01806
[372]	eval-error:0.16209	train-error:0.01802
[373]	eval-error:0.16112	train-error:0.01799
[374]	eval-error:0.16151	train-error:0.01799
[375]	eval-error:0.16161	train-error:0.01797
[376]	eval-error:0.16161	train-error:0.01797
[377]	eval-error:0.16170	train-error:0.01797
[378]	eval-error:0.16180	train-error:0.01793
[379]	eval-error:0.16180	train-error:0.01794
[380]	eval-error:0.16190	train-error:0.01788
[381]	eval-error:0.16170	train-error:0.01790
[382]	eval-error:0.16248	train-error:0.01788
[383]	eval-error:0.16278	train-error:0.01787
[384]	eval-error:0.16297	train-error:0.01787
[385]	eval-error:0.16287	train-error:0.01788
[386]	eval-error:0.16229	train-error:0.01788
[387]	eval-error:0.16209	train-error:0.01787
[388]	eval-error:0.16209	train-error:0.01785
[389]	eval-error:0.16190	train-error:0.01784
[390]	eval

In [100]:
df_UID_MID_test["prediction"] = y_test_pred

seq_identity = ["60-80%", "40-60%", "<40%"]

for identity in seq_identity:
    y_true = np.array(df_UID_MID_test["outcome"].loc[df_UID_MID_test["Sequence identity"] == identity])
    y_pred = np.array(df_UID_MID_test["prediction"].loc[df_UID_MID_test["Sequence identity"] == identity])
    acc = np.mean(y_pred == np.array(y_true))
    mcc = matthews_corrcoef(np.array(y_true), y_pred)
    print("Sequence identity %s, Accuracy: %s, MCC: %s \n" % (identity, acc, mcc))

Sequence identity 60-80%, Accuracy: 0.8461873638344226, MCC: 0.6140625864438229 

Sequence identity 40-60%, Accuracy: 0.8399518652226233, MCC: 0.5777100133860964 

Sequence identity <40%, Accuracy: 0.8193774660236738, MCC: 0.5039163841394687 

